<center><b>DIGHUM101</b></center>
<center>3-3: PRAW and Tweepy (Reddit and Twitter APIs)</center>

---

# Introduction to Social Media Data

Data from social media platforms are more important than ever. However, critical analysis and fact-checking are also more important than ever as misinformation, hatespeech, bots, and trolling become more and more prevalent. [Reddit](https://www.reddit.com/) and [Twitter](https://twitter.com/) are two of the most popular social media websites with useful public APIs.

Note that [DIGHUM160](http://guide.berkeley.edu/courses/dighum/) consists of more extensive critical research on Reddit.

In [ ]:
# Library installations

#!pip install praw
#!pip install tweepy

In [ ]:
# Library imports

# Python Reddit API Wrapper
import praw

# Twitter API client
import tweepy

import pandas as pd 

# Reddit

Let's follow this tutorial to see how you can connect to Reddit's [API](https://en.wikipedia.org/wiki/Application_programming_interface): 

https://pythonprogramming.net/introduction-python-reddit-api-wrapper-praw-tutorial/

[The praw documentation is also very useful](https://praw.readthedocs.io/en/latest/getting_started/quick_start.html)!

1. Visit https://www.reddit.com/
2. Click "Sign Up/Log In" to create an account or login
3. Go to https://old.reddit.com/prefs/apps/
4. Click "Create an ap" (at the bottom of the page)
5. Click the "script" radio button (**not** a "web app" or "installed app"!)
6. Give your project a name and description
7. Enter an "about URL" if you choose (such as your project name)
8. Enter http://localhost:8080 for your the "redirect uri"

# 1. Authenticate!

Create an instance and add your information: 

- client_id = the code under your project name in the upper-left
- client_secret = your API access token
- password = your password
- user_agent = put something like: 'dhe 1.0 by /u/dh_example'
- username = your username

![reddit](../../Img/reddit-API.png)

# Create a Reddit API instance

In [ ]:
# We need these 5 things - let's overwrite them with our own!

reddit = praw.Reddit(client_id='YOUR_CLIENT_ID',
                     client_secret='YOUR_CLIENT_SECRET',
                     password='YOUR_REDDIT_PSW',
                     user_agent='dh by /u/YOUR_USERNAME', 
                     username='YOUR_USERNAME'
                    )

In [ ]:
# Check out a subreddit
subreddit = reddit.subreddit('BlackLivesMatter')

In [ ]:
subreddit.description_html

In [ ]:
# Filter the Python subreddit by top-scoring topics
hot_blm = subreddit.top()
hot_blm

In [ ]:
# Check out reddit methods
#reddit.

In [ ]:
# Check out subreddit methods - you can also get hot topics, controversial topics, etc.
subreddit.

In [ ]:
# Iterate over the "hot" submissions to get the object IDs
for submission in hot_blm:
    print(submission.id)

In [ ]:
subreddit.

In [ ]:
# Return just the first 5 submissions and print their selftext
hot_blm = subreddit.hot(limit = 5)
for submission in hot_blm:
    print(submission.selftext)

In [ ]:
# Define a blank dictionary to store the metadata
conversedict = {}

# Get more information
hot_blm = subreddit.top(limit = 5)
for submission in hot_blm:
    if not submission.stickied:
        print('title: {}, created: {}, score: {}, ratio: {}'.format(submission.title,
                                                                submission.created,
                                                                submission.score,
                                                                submission.upvote_ratio))
        
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            if comment.id not in conversedict:
                conversedict[comment.id] = [comment.body,{}]
                if comment.parent() != submission.id:
                    parent = str(comment.parent())
                    conversedict[parent][1][comment.id] = [comment.ups, comment.body]

In [ ]:
# Define a blank list for export to data frame
reddit_output = []

for post_id in conversedict:
    submission = conversedict[post_id][0]
    comments = conversedict[post_id][1]
    if len(comments) > 1:
        print(f'Submission: {submission}')
        for comment in comments:
            reddit_output.append(comments[comment])
            print(f'Comments: {comments[comment]}')
        print(35*'_')

In [ ]:
# View the output of the variable
reddit_output

# Convert to data frame

In [ ]:
reddit_df = pd.DataFrame(reddit_output, columns = ["Upvotes", "Text"])
reddit_df.head()

In [ ]:
# Save original message as file name...
reddit_df.to_csv("blm_reddit.csv")

Keep in mind that you probably will want more data from this API than just submissions and comments with their upvotes! The Reddit API allows access to a lot more metadata – submission dates, "Flair" (a way of categorizing data that works differently for each community), and so on. It all depends on the research question you're asking. But in general, when collecting data from an API, it's worthwhile to not cut any corners. 

# Twitter

Twitter works similarly, but you have to fill out more information to get permission to use their API. 

Here is a nice Tweepy walkthrough: 

https://realpython.com/twitter-bot-python-tweepy/

[The Tweepy docs are also very useful!](http://docs.tweepy.org/en/latest/)

1. Visit the Twitter Developer site: https://developer.twitter.com/en and create an account/login
2. Navigate to the Developer Portal
3. Under Projects & Apps, create a new project (note down the API codes).
4. Create a new app within the project and generate access keys (note down the API codes).


# Authenticate!

In [ ]:
# Similar to reddit! 
auth = tweepy.OAuthHandler("API_KEY", "API_SECRET")
auth.set_access_token("ACCESS_TOKEN", "ACCESS_TOKEN_SECRET")

# What do these arguments do?
api = tweepy.API(auth, wait_on_rate_limit=True)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

In [ ]:
# Sanity check - did you validate correctly?
try:
    api.verify_credentials()
    print("Success!")
except:
    print("Invalid authentication")

In [ ]:
# tweepy methods!
# tweepy.

In [ ]:
# Maybe the Internet can provide better explanations?
#tweepy.API?

In [ ]:
# Find followers
user = api.get_user(screen_name="BarackObama")

print("User details:")
print(user.name)
print(user.description)
print(user.location)

print("Last 20 Followers:")
for follower in user.followers():
    print(follower.name)

In [ ]:
# Get the User object for a twitter handle...
user = api.get_user(screen_name='BarackObama')

In [ ]:
# Define an empty list

def tweets(user_name):
          
    # Authorize yourself
    # consumer_key, consumer_secret
    auth = tweepy.OAuthHandler("API_KEY", "API_SECRET")

    # Provide your tokens
    # access_key, access_secret
    auth.set_access_token("ACCESS_TOKEN", "ACCESS_TOKEN_SECRET")

    # Define an API instance
    api = tweepy.API(auth)
  
    # Define an empty list to store the tweets
    storage = []

    # Get 20 tweets
    num_tweets = 20
    tweets = api.user_timeline(screen_name = user_name)
  
    # Return user, tweet, date and time, and body
    tweet_data = [tweet.text for tweet in tweets]
    for i in tweet_data: 
  
        # Append storage
        storage.append(i)

    # What to return?
    return storage

In [ ]:
twitter_output = tweets("BarackObama")
print(twitter_output)

In [ ]:
# Convert to DataFrame
twitter_df = pd.DataFrame(twitter_output, columns = ["Tweet"])
twitter_df.head()

In [ ]:
twitter_df.to_csv("Obama_tweets.csv")

In [ ]:
MAX_TWEETS = 10

blm_tweets = []

for tweet in tweepy.Cursor(api.search_tweets, q='#BLM').items(MAX_TWEETS):
    blm_tweets.append(tweet.text)

In [ ]:
blm_tweets